In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

In [2]:
# Setting up the VGG19  model for transfer learning
vgg= VGG19(include_top= False, weights= "imagenet", input_shape= (224, 224, 3))

In [3]:
# Freezing all the layers
for layer in vgg.layers:
    layer.trainable = False

In [4]:
# 
x= Flatten()(vgg.output)

In [5]:
pred= Dense(4, activation= "softmax")(x)

In [6]:
model= tf.keras.Model(vgg.inputs, pred)

In [7]:
model.compile(loss= "categorical_crossentropy", optimizer= "adam", metrics= ["acc"])

In [8]:
train_generator= ImageDataGenerator(rescale=1./255,  shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_generator= ImageDataGenerator(rescale=1./255,  shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [9]:
train_data = train_generator.flow_from_directory( '../input/cotton-disease-dataset/Cotton Disease/train', target_size=(224, 224),
        batch_size=64)

In [10]:
test_data = test_generator.flow_from_directory( '../input/cotton-disease-dataset/Cotton Disease/test', target_size=(224, 224),
        batch_size=64)

In [ ]:
es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience = 3)

In [11]:
h= model.fit(train_data, steps_per_epoch= len(train_data), epochs=10, validation_data= test_data, validation_steps= len(test_data), callbacks=[es_cb])

In [13]:
model.save_weights("cotton.h5")

In [18]:
labels = train_data.class_indices

In [20]:
with open("labels.json", "w") as l:
    json.dump(labels, l)